In [1]:
from coffea.nanoevents import NanoAODSchema,NanoEventsFactory
from coffea import processor
from coffea.analysis_tools import PackedSelection
import hist
from Snip_debug import *
import awkward as ak
import numpy as np

In [2]:
events = NanoEventsFactory.from_root(
    "/commonfs/store/data/Run2018B/MET/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v2/250000/7B7A01B5-FE11-5944-B73B-1FF6CB58CB83.root",
    schemaclass=NanoAODSchema.v7,
    metadata={
        "dataset":"MET_Run2018"
    }
).events()

In [3]:
eventsMC = NanoEventsFactory.from_root(
    "/commonfs/store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/0520A050-AF68-EF43-AA5B-5AA77C74ED73.root",
    schemaclass=NanoAODSchema.v7,
    metadata={
        "dataset":"TTToSemiLeptonic"
    }
).events()

In [4]:
cutflow={}
cutflow["No cuts"] = len(events)

In [5]:
fraction_of_affected = 0.647724485 #Obtained by Shivani from Brilcalc
fraction_of_unaffected = 1 - fraction_of_affected
print(len(events.MET.pt))
pass_bool_array = np.ones(round(len(events.MET.pt)*fraction_of_affected),dtype=bool) #We use events.MET.pt to be sure that no None contribution to length happens
block_bool_array = ~np.ones(round(len(events.MET.pt)*fraction_of_unaffected),dtype=bool)
print(len(pass_bool_array))
print(len(block_bool_array))
print(len(pass_bool_array)+len(block_bool_array))
block_bool_array

2172178
1406973
765205
2172178


array([False, False, False, ..., False, False, False])

In [6]:
fake_run_mask = np.concatenate((pass_bool_array,block_bool_array))
fake_run_mask

array([ True,  True,  True, ..., False, False, False])

In [7]:
ak.Array(fake_run_mask)

<Array [True, True, True, ... False, False] type='2172178 * bool'>

In [8]:
def HEM_veto_bril_test(events, cutflow):
    """
    Developed from Shivani's implementation of HEM veto.
    """
    dataset = events.metadata["dataset"]
    is_data = dataset.startswith("MET")
    if is_data == True :
        mask = (events.run >= 319077)
        passes_hem = ak.all((events.Jet.phi < -1.57) | (events.Jet.phi > -0.87) | (events.Jet.eta < -3.0) | (events.Jet.eta > -1.3), axis=1)
        HEM_region = mask & passes_hem
        Non_HEM_region = events.run < 319077
        events = events[Non_HEM_region | HEM_region]
    elif is_data == False :
        # MC should mimic the ratios of affected and unaffected events
        fraction_of_affected = 0.647724485 #Obtained by Shivani from Brilcalc
        fraction_of_unaffected = 1 - fraction_of_affected
        #Create a fake run cut mask just like in data, by using this ratio
        pass_bool_array = np.ones(round(len(events.MET.pt)*fraction_of_affected),dtype=bool) #We use events.MET.pt to be sure that no None contribution to length happens
        block_bool_array = ~np.ones(round(len(events.MET.pt)*fraction_of_unaffected),dtype=bool)
        print(pass_bool_array, len(pass_bool_array))
        print(block_bool_array, len(block_bool_array))
        fake_run_mask = np.concatenate((pass_bool_array,block_bool_array))
        print(fake_run_mask, len(fake_run_mask))
        passes_hem = ak.all((events.Jet.phi < -1.57) | (events.Jet.phi > -0.87) | (events.Jet.eta < -3.0) | (events.Jet.eta > -1.3), axis=1)
        HEM_region = fake_run_mask & passes_hem
        print(HEM_region)
        Non_HEM_region = ~fake_run_mask
        print(Non_HEM_region)
        events = events[Non_HEM_region | HEM_region]
    else :
        raise  Exception("Function HEM_veto_bril_test caught an error.")
    cutflow["HEM veto"] = len(events)
    return events, cutflow

In [9]:
cutflow={}
cutflow["No cuts"] = len(events)
a, b = HEM_veto_bril_test(events, cutflow)

In [10]:
b

{'No cuts': 2172178, 'HEM veto': 2167091}

In [11]:
cutflow={}
cutflow["No cuts"] = len(eventsMC)
a, b = HEM_veto_bril_test(eventsMC, cutflow)

[ True  True  True ...  True  True  True] 838155
[False False False ... False False False] 455845
[ True  True  True ... False False False] 1294000
[True, True, True, True, True, True, ... False, False, False, False, False, False]
[False False False ...  True  True  True]


In [12]:
b

{'No cuts': 1294000, 'HEM veto': 1163578}

In [14]:
events.fields

['L1PreFiringWeight',
 'PuppiMET',
 'fixedGridRhoFastjetCentralCalo',
 'SoftActivityJetNjets10',
 'SoftActivityJetNjets2',
 'FsrPhoton',
 'run',
 'SubJet',
 'OtherPV',
 'SoftActivityJetNjets5',
 'event',
 'DeepMETResponseTune',
 'HLT',
 'IsoTrack',
 'RawMET',
 'HLTriggerFirstPath',
 'TrigObj',
 'fixedGridRhoFastjetCentralChargedPileUp',
 'Muon',
 'boostedTau',
 'Tau',
 'luminosityBlock',
 'fixedGridRhoFastjetAll',
 'HLTriggerFinalPath',
 'ChsMET',
 'RawPuppiMET',
 'SoftActivityJetHT5',
 'fixedGridRhoFastjetCentral',
 'L1',
 'FatJet',
 'MET',
 'DeepMETResolutionTune',
 'L1Reco',
 'CorrT1METJet',
 'SoftActivityJetHT',
 'fixedGridRhoFastjetCentralNeutral',
 'TkMET',
 'SV',
 'Proton',
 'SoftActivityJet',
 'Photon',
 'SoftActivityJetHT10',
 'PPSLocalTrack',
 'Electron',
 'PV',
 'LowPtElectron',
 'Flag',
 'Jet',
 'SoftActivityJetHT2',
 'CaloMET']

In [22]:
events.Jet.__doc__

'slimmedJets, i.e. ak4 PFJets CHS with JECs applied, after basic selection (pt > 15)'

In [24]:
events.FatJet.__doc__

'slimmedJetsAK8, i.e. ak8 fat jets for boosted analysis'

In [27]:
events.FatJet.fields

['area',
 'btagCSVV2',
 'btagDDBvLV2',
 'btagDDCvBV2',
 'btagDDCvLV2',
 'btagDeepB',
 'btagHbb',
 'deepTagMD_H4qvsQCD',
 'deepTagMD_HbbvsQCD',
 'deepTagMD_TvsQCD',
 'deepTagMD_WvsQCD',
 'deepTagMD_ZHbbvsQCD',
 'deepTagMD_ZHccvsQCD',
 'deepTagMD_ZbbvsQCD',
 'deepTagMD_ZvsQCD',
 'deepTagMD_bbvsLight',
 'deepTagMD_ccvsLight',
 'deepTag_H',
 'deepTag_QCD',
 'deepTag_QCDothers',
 'deepTag_TvsQCD',
 'deepTag_WvsQCD',
 'deepTag_ZvsQCD',
 'eta',
 'mass',
 'msoftdrop',
 'n2b1',
 'n3b1',
 'particleNetMD_QCD',
 'particleNetMD_Xbb',
 'particleNetMD_Xcc',
 'particleNetMD_Xqq',
 'particleNet_H4qvsQCD',
 'particleNet_HbbvsQCD',
 'particleNet_HccvsQCD',
 'particleNet_QCD',
 'particleNet_TvsQCD',
 'particleNet_WvsQCD',
 'particleNet_ZvsQCD',
 'particleNet_mass',
 'phi',
 'pt',
 'rawFactor',
 'tau1',
 'tau2',
 'tau3',
 'tau4',
 'lsf3',
 'jetId',
 'subJetIdx1',
 'subJetIdx2',
 'electronIdx3SJ',
 'muonIdx3SJ',
 'nConstituents',
 'subJetIdx1G',
 'subJetIdx2G',
 'subJetIdxG']

In [29]:
events.FatJet.btagDDBvLV2

<Array [[], [0.000127, 0.00125], ... [], []] type='2172178 * var * float32[param...'>

In [37]:
#events.caches[0].keys()

In [39]:
events.Jet.pt

<Array [[131, 60, 26.5, ... 15.8, 15.7, 15.7]] type='2172178 * var * float32[par...'>

In [45]:
events['Jet','special'] = events.Jet.pt** - events.Jet.pt
events.Jet.special

<Array [[0, 0, ... 1.86e-19, 1.92e-19]] type='2172178 * var * float32'>

In [50]:
print(events.Jet.special.__doc__)


    Args:
        data (#ak.layout.Content, #ak.partition.PartitionedArray, #ak.Array, `np.ndarray`, `cp.ndarray`, `pyarrow.*`, str, dict, or iterable):
            Data to wrap or convert into an array.
               - If a NumPy array, the regularity of its dimensions is preserved
                 and the data are viewed, not copied.
               - CuPy arrays are treated the same way as NumPy arrays except that
                 they default to `kernels="cuda"`, rather than `kernels="cpu"`.
               - If a pyarrow object, calls #ak.from_arrow, preserving as much
                 metadata as possible, usually zero-copy.
               - If a dict of str → columns, combines the columns into an
                 array of records (like Pandas's DataFrame constructor).
               - If a string, the data are assumed to be JSON.
               - If an iterable, calls #ak.from_iter, which assumes all dimensions
                 have irregular lengths.
        behavior (None or d